In [2]:
from utils import commun_functions, data, diagram, GurobiSolver 
from RecuitSimule import RS
from VariableNeighborhoodSearch import VNS
#from GurobiSolver import GurSol
import random
import math
import matplotlib.pyplot as plt
from PIL import Image
import time 

In [3]:
instancefilename='Instances/Kacem1.fjs'
ProcTime=commun_functions.FJSInstanceReading(instancefilename)
data_instance = data.data(0.0, 0.0, 0, ProcTime)

In [5]:
kmax=sum([1  for jid,job in enumerate(ProcTime) for opid,op in enumerate(job) if len(op)>1])
UBcmax=sum([ ])
NMachines=max([om[0]  for jid,job in enumerate(ProcTime) for opid,op in enumerate(job) for omid,om in enumerate(op)])
InstanceAuthorName='Kacem'
instancename='Kacem2'
resfilename="Results/%s.csv" % instancename
resfich = open(resfilename, "a")
resfich.write("n;mu;lambda;pmtime; cmax")
for m in range(NMachines):
    resfich.write("; Nbr%d" % m)
resfich.write("\n")
MUs=[0.1]
EHFs=[0.8]
PMTIMEs=[2]
N=len(MUs)*len(EHFs)*len(PMTIMEs)
n=0
nbrestart=3
maxtime=10
showplots=True
for mu in MUs:
    data_instance.mu=mu
    for EHFmax in EHFs:
        data_instance.lambdaPM=EHFmax
        for pmtime in PMTIMEs: 
            data_instance.PM_time=pmtime
            VNS_Algo = VNS(kmax, maxtime, data_instance)
            c0=float("inf")
            best_energy=c0
            for k in range(nbrestart): 
                RS_initial_solution = commun_functions.GenererSolution(data_instance)
                RS_instance = RS(RS_initial_solution, 50, 0.1, 0.1, 10, 100)
                best_sub_solution, best_sub_energy, nb_sub_iteration = RS_instance.simulated_annealing(data_instance)
                if best_sub_energy < best_energy : 
                    best_solution = best_sub_solution
                    best_energy = best_sub_energy
                    nb_iteration = nb_sub_iteration
                a,b,c,d,e,f,solVNS = VNS_Algo.BasicVNS()
                if c<c0:
                    a0,b0,c0,d0,e0,f0,bestsolVNS = a,b,c,d,e,f,solVNS
            print('bestVNS=',c0,' BestRS=',best_energy)
            """"
            pngfilename='results/Figs/%s/%s_%d.png' % (InstanceAuthorName,instancename,n)
            GanttVNS = diagram.diagram(a0,b0,pmtime,mu,c0,d0,e0,f0,pngfilename)
            GanttVNS.plotGantt()
            GanttVNS.plotEHF2()
            nbrmaint=[len(mtasks) for mi,mtasks in enumerate(e0)]
            resfich.write("%d ;%d ; %d ; %d ; %d" % (n,mu,EHFmax,pmtime,c0))
            for m in range(len(nbrmaint)):
                resfich.write("; %d" % nbrmaint[m])
            resfich.write("\n")
            print(f"Best solution = {bestsolVNS}")
            if showplots:
                # Charger les images enregistrées
                image_gantt = Image.open(GanttVNS.ganttsavefilename)
                image_ehf = Image.open(GanttVNS.ehfplotsavefilename)
                # Afficher les deux images dans une seule fenêtre
                fig, ax = plt.subplots(1, 2, figsize=(20, 10))
                ax[0].imshow(image_gantt)
                ax[0].axis('off')
                ax[0].set_title('Gantt Chart')
                ax[1].imshow(image_ehf)
                ax[1].axis('off')
                ax[1].set_title('EHF Chart')
                plt.show()
            """
resfich.close()

Initial solution:  [(1, 0), (0, 1), (0, 3), (1, 4), (2, 4), (3, 0), (3, 2), (1, 0), (2, 3), (2, 0), (2, 3), (0, 1)]
*********** Basic VNS with kmax= 12  tmax= 10 ***********
Initial solution:  [(2, 0), (1, 3), (1, 2), (2, 2), (3, 3), (0, 4), (0, 3), (2, 4), (3, 0), (0, 3), (2, 4), (1, 4)]
*********** Basic VNS with kmax= 12  tmax= 10 ***********
Initial solution:  [(0, 1), (3, 2), (3, 2), (2, 3), (2, 4), (1, 3), (1, 0), (0, 4), (2, 0), (1, 1), (0, 0), (2, 0)]
*********** Basic VNS with kmax= 12  tmax= 10 ***********
bestVNS= 13  BestRS= 26


In [1]:
from utils import commun_functions, data, diagram, GurobiSolver 
from RecuitSimule import RS
from VariableNeighborhoodSearch import VNS
#from GurobiSolver import GurSol
import random
import math
import matplotlib.pyplot as plt
from PIL import Image
import time 
instancefilename='Instances/Kacem1.fjs'
ProcTime=commun_functions.FJSInstanceReading(instancefilename)
data_instance = data.data(0.0, 0.0, 0, ProcTime)
data_instance.mu=0.01
data_instance.lambdaPM=0.9
data_instance.PM_time=2
RandomSol=[(jid,mo[0]) for jid,job in enumerate(ProcTime) for opid,op in enumerate(job) for mo in random.sample(op,1)]
RandomSol=random.sample(RandomSol,len(RandomSol))
#print(RandomSol)
#for jid,job in enumerate(ProcTime): print(job)
NM,NJ,UBcmax,schedule,maint,ehf=commun_functions.evaluate(RandomSol,data_instance)
OPTSOL=GurobiSolver.GurSol(data_instance.mu, data_instance.lambdaPM, data_instance.lambdaQ, data_instance.lambdaC, data_instance.PM_time, data_instance.CM_time, ProcTime)
X, Y, ZZ, DELTA, objf, gap, cputime, start_time, task_duration, assigned_machine=OPTSOL.SolveIPQM_FJSP(UBcmax,1,0,False)


TypeError: Cannot interpret '4' as a data type

In [7]:
import numpy as np
PRT=[]
NOP=[3,2,7]
for j in range(3):
    PRT[j]=[]
    for i in range(NOP[j]):
        PRT[j][i]=[]   
        for m in range(NM):
            PRT[j][i][m]=0
print(PRT)


IndexError: list assignment index out of range

# Recuit Simulé 

In [5]:
initial_solution = commun_functions.GenererSolution(data_instance)
RS_instance = RS(initial_solution, 50, 0.1, 0.1, 100, 100)
best_solution, best_energy, nb_iteration = RS_instance.simulated_annealing(data_instance)

iterations = 100
x = [i for i in range(iterations)]

solutions = []
for i in range(iterations): 
    initial_solution = commun_functions.GenererSolution(data_instance)
    RS_instance = RS(initial_solution, 50, 0.1, 0.1, 10, 100)

    best_sub_solution, best_sub_energy, nb_sub_iteration = RS_instance.simulated_annealing(data_instance)
    
    solutions.append(best_energy)

    if best_sub_energy < best_energy : 
        best_solution = best_sub_solution
        best_energy = best_sub_energy
        nb_iteration = nb_sub_iteration


print("intial Solution:", initial_solution)
print("Best Solution:", best_solution, "Best energy:", best_energy)
print("Number of iterations:", nb_iteration)
print("Solutions ", solutions)

Initial solution:  [(0, 1), (3, 2), (0, 0), (2, 4), (3, 0), (2, 0), (2, 0), (1, 0), (1, 3), (0, 1), (1, 0), (2, 3)]
Initial solution:  [(3, 0), (2, 2), (1, 2), (0, 3), (1, 0), (3, 4), (1, 4), (2, 1), (2, 3), (0, 0), (2, 4), (0, 4)]
Initial solution:  [(1, 1), (0, 3), (0, 3), (1, 4), (2, 2), (0, 1), (2, 4), (1, 0), (2, 0), (3, 3), (2, 3), (3, 1)]
Initial solution:  [(2, 4), (0, 2), (2, 0), (1, 4), (1, 0), (0, 0), (1, 0), (2, 0), (3, 3), (3, 1), (2, 0), (0, 4)]
Initial solution:  [(1, 0), (1, 3), (0, 2), (3, 1), (2, 0), (2, 2), (2, 3), (0, 3), (0, 2), (3, 0), (1, 3), (2, 0)]
Initial solution:  [(0, 3), (1, 1), (2, 2), (3, 2), (2, 1), (0, 0), (1, 0), (0, 2), (1, 4), (2, 3), (3, 4), (2, 3)]
Initial solution:  [(2, 1), (1, 4), (0, 4), (3, 4), (1, 2), (2, 3), (2, 2), (3, 3), (0, 3), (2, 2), (1, 4), (0, 3)]
Initial solution:  [(2, 0), (1, 3), (1, 3), (0, 1), (0, 2), (0, 1), (3, 2), (3, 2), (1, 3), (2, 1), (2, 0), (2, 2)]
Initial solution:  [(2, 3), (1, 0), (0, 4), (0, 3), (2, 1), (3, 2), (1, 